In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from hs2 import herdingspikes

%matplotlib inline

In [ ]:
data_path = 'data/rawDataSample.bin'
# data = h5py.File(data_path, 'r')
# data = np.fromfile(data_path, dtype=np.int16).reshape((1800000,385))

H = herdingspikes(datapath=data_path)

In [ ]:
# detectData(data, neighbours, spikefilename, shapefilename, channels, sfd, thres, maa = None, maxsl = None, minsl = None, ahpthr = None, tpre = 1, tpost = 2)
# MinAvgAmp minimal avg. amplitude of peak (in units of Qd)
# MaxSl dead time in frames after peak, used for further testing
# MinSl length considered for determining avg. spike amplitude
# AHPthr signal should go below that threshold within MaxSl-Slmin frames

num_channels = 385
num_recording_channels = 374
spike_delay = 5
spike_peak_duration = 5
noise_duration = 2
noise_amp = 80000
max_neighbors = 14
to_localize = True
cutout_length = 40
fps = 30000
threshold = 20 #20

H.DetectFromRaw(num_channels, num_recording_channels, spike_delay,
               spike_peak_duration, noise_duration, noise_amp, max_neighbors,
               to_localize, cutout_length, fps, threshold, maa=0,
               maxsl=12, minsl=3, ahpthr=0)

# H.LoadDetected()

In [ ]:
H.PlotTracesChannels(1008)

In [ ]:
plt.figure(figsize=(10, 4))
H.PlotAll(invert=True, s=2, alpha=0.5)

In [ ]:
H.CombinedMeanShift(bandwidth=20, alpha=40, bin_seeding=True,
                    min_bin_freq=10, pca_ncomponents=2, pca_whiten=True,
                    n_jobs=-1)

In [ ]:
plt.figure(figsize=(10, 5))
H.PlotAll(invert=True, s=10, show_labels=True)
plt.xlim((1350,1600))


In [ ]:
units = (121,4)

H.PlotShapes(units)